### Flash Card Data To Firebase

In [ ]:
#!pip install google-cloud-storage

In [ ]:
#!pip install firebase_admin

In [ ]:
#!pip install firebase

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
#!pip install pyrebase4

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [ ]:
#import pyrebase
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import shutil

In [ ]:
#import firebase_admin
#from firebase_admin import credentials

cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")
app = firebase_admin.initialize_app(cred)

In [ ]:
fire_db = firestore.client()

#### Read Data

In [ ]:
data_root = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Root_Word_Translate_99.xlsx")
data_root

In [ ]:
data_word = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Word_Translate_300.xlsx")
data_word.reset_index(inplace=True)
data_word.rename(columns={"index":"sequence"}, inplace=True)
data_word["sequence"] = data_word["sequence"].apply(lambda x: x+1)
data_word

In [ ]:
data_twogram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Twogram_Translate_Frequency.xlsx")
data_twogram.reset_index(inplace=True)
data_twogram.rename(columns={"index":"sequence"}, inplace=True)
data_twogram["sequence"] = data_twogram["sequence"].apply(lambda x: x+1)
data_twogram

In [ ]:
data_threegram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Threegram_Translate_Frequency.xlsx")
data_threegram.reset_index(inplace=True)
data_threegram.rename(columns={"index":"sequence"}, inplace=True)
data_threegram["sequence"] = data_threegram["sequence"].apply(lambda x: x+1)
data_threegram

In [ ]:
data_ayah = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Plain_Ayahs_Frequency.xlsx")
data_ayah.reset_index(inplace=True)
data_ayah.rename(columns={"index":"sequence"}, inplace=True)
data_ayah["sequence"] = data_ayah["sequence"].apply(lambda x: x+1)
data_ayah

In [ ]:
data_ayah.to_csv("Test.csv", index=False)

#### Sending Data To Firestore

##### Root Data

In [ ]:
data_root = data_root.iloc[0:15000,]  # data divide because of quota
data_root

In [ ]:
# root
sequence_list = data_root["sequence"].to_list()
root_arabic_list = data_root["root_arabic"].to_list()
root_list = data_root["root"].to_list()
root_arabic_freq_list = data_root["root_arabic_frequency"].to_list()
# word 
arabic_simple_list = data_root["arabic_simple"].to_list()
arabic_list = data_root["arabic"].to_list()
trans_eng_list = data_root["translate_english"].to_list()
trans_urdu_list = data_root["translate_urdu"].to_list()
trans_hindi_list = data_root["translate_hindi"].to_list()
trans_indo_list = data_root["translate_indonesian"].to_list()
trans_bangla_list = data_root["translate_bangla"].to_list()
trans_tur_list = data_root["translate_turkish"].to_list()
trans_rus_list = data_root["translate_russian"].to_list()
surah_list = data_root["surah"].to_list()
ayah_list = data_root["ayah"].to_list()
word_rank_list = data_root["word rank"].to_list()

In [ ]:
firestore_sending_root_data = zip(sequence_list,root_arabic_list,root_list,root_arabic_freq_list,arabic_simple_list,arabic_list,surah_list,ayah_list,word_rank_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [ ]:
# Sending Root Data
for sequence,root_arabic,root,root_arabic_freq,arabic_simple,arabic,surah,ayah,word_rank,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_root_data:
    data = {"root_arabic":root_arabic,
            "root":root,
            "root_arabic_freq":root_arabic_freq,
            "arabic_simple":arabic_simple,
            "arabic":arabic,
            "surah":surah,
            "ayah":ayah,
            "word_rank":word_rank,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Root_Data").document(f"root{sequence}").collection(f"{root}").document(f"{arabic_simple}").set(data)  # Not: Eger document ile bitiyorsa data set ile gönderiliyor.
    #fire_db.collection("Root_Data").document(f"root{sequence}").collection("Detail").add(data)  # Not: Eğer collection ile bitiyorsa data add ile gönderiliyor ve collection dan sonraki
                                                                                         # document adları rastgele oluşturuluyor.                                                                                               

In [ ]:
#fire_db.collection(u'cities').document(u'DC').delete()

##### Word Data

In [ ]:
data_word

In [ ]:
# word
sequence_list = data_word["sequence"].to_list()
arabic_simple_list = data_word["arabic_simple"].to_list()
arabic_simple_freq_list = data_word["arabic_simple_frequency"].to_list()
trans_eng_list = data_word["translate_english"].to_list()
trans_urdu_list = data_word["translate_urdu"].to_list()
trans_hindi_list = data_word["translate_hindi"].to_list()
trans_indo_list = data_word["translate_indonesian"].to_list()
trans_bangla_list = data_word["translate_bangla"].to_list()
trans_tur_list = data_word["translate_turkish"].to_list()
trans_rus_list = data_word["translate_russian"].to_list()

In [ ]:
firestore_sending_word_data = zip(sequence_list,arabic_simple_list,arabic_simple_freq_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [ ]:
# Sending Word Data
for sequence,arabic_simple,arabic_simple_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_word_data:
    data = {"arabic_simple":arabic_simple,
            "arabic_simple_freq":arabic_simple_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Word_Data").document(f"word{sequence}").collection(f"{arabic_simple}").document("detail").set(data)

##### Twogram Data

In [ ]:
data_twogram

In [ ]:
# twogram
sequence_list = data_twogram["sequence"].to_list()
twogram_string_list = data_twogram["twogram_string"].to_list()
twogram_string_freq_list = data_twogram["twogram_string_frequency"].to_list()
twogram_trans_eng_list = data_twogram["twogram_english"].to_list()
twogram_trans_urdu_list = data_twogram["twogram_urdu"].to_list()
twogram_trans_hindi_list = data_twogram["twogram_hindi"].to_list()
twogram_trans_indo_list = data_twogram["twogram_indonesian"].to_list()
twogram_trans_bangla_list = data_twogram["twogram_bangla"].to_list()
twogram_trans_tur_list = data_twogram["twogram_turkish"].to_list()
twogram_trans_rus_list = data_twogram["twogram_russian"].to_list()

In [ ]:
firestore_sending_twogram_data = zip(sequence_list,twogram_string_list,twogram_string_freq_list,twogram_trans_eng_list,twogram_trans_urdu_list,twogram_trans_hindi_list,twogram_trans_indo_list,twogram_trans_bangla_list,twogram_trans_tur_list,twogram_trans_rus_list)

In [ ]:
# Sending Twogram Data
for sequence,twogram,twogram_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_twogram_data:
    data = {"twogram_simple":twogram,
            "twogram_freq":twogram_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Twogram_Data").document(f"twogram{sequence}").collection(f"{twogram}").document("detail").set(data)

##### Threegram Data

In [ ]:
data_threegram

In [ ]:
# threegram
sequence_list = data_threegram["sequence"].to_list()
threegram_string_list = data_threegram["threegram_string"].to_list()
threegram_string_freq_list = data_threegram["threegram_string_frequency"].to_list()
threegram_trans_eng_list = data_threegram["threegram_english"].to_list()
threegram_trans_urdu_list = data_threegram["threegram_urdu"].to_list()
threegram_trans_hindi_list = data_threegram["threegram_hindi"].to_list()
threegram_trans_indo_list = data_threegram["threegram_indonesian"].to_list()
threegram_trans_bangla_list = data_threegram["threegram_bangla"].to_list()
threegram_trans_tur_list = data_threegram["threegram_turkish"].to_list()
threegram_trans_rus_list = data_threegram["threegram_russian"].to_list()

In [ ]:
firestore_sending_threegram_data = zip(sequence_list,threegram_string_list,threegram_string_freq_list,threegram_trans_eng_list,threegram_trans_urdu_list,threegram_trans_hindi_list,threegram_trans_indo_list,threegram_trans_bangla_list,threegram_trans_tur_list,threegram_trans_rus_list)

In [ ]:
# Sending Threegram Data
for sequence,threegram,threegram_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_threegram_data:
    data = {"threegram_simple":threegram,
            "threegram_freq":threegram_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Threegram_Data").document(f"threegram{sequence}").collection(f"{threegram}").document("detail").set(data)

##### Ayah Data

#### Test

In [ ]:
data = {"name":"Ertugrul","surname":"Kurubal"}

In [ ]:
fire_db.collection("Root_Data").document("Person").collection("Detail").add(data)

In [ ]:
fire_db.collection("Root_Data2").document("Person2").set(data)

In [ ]:
# delete firestore
fire_db.collection(u"Root_Data").document(u"Person").delete()

In [ ]:
fire_db.collection(u"Root_Data").document(u"Person").update({})

#### Realtime Database Web App

In [ ]:
import pyrebase  # pyrebase running on colab. Tested in colab
import pandas as pd

In [ ]:
print(pyrebase.__version__)

In [ ]:
firebaseConfig = {
    "apiKey": "xxxxxaSyAKKZn1txxxxx",
    "authDomain": "xxxxxx.firebaseapp.com",
    "databaseURL": "https://flashxxxxx",
    "projectId": "flxxxx",
    "storageBucket": "flxxxxxspot.com",
    "messagingSenderId": "71xxxx",
    "appId": "1xxxxxx2277a217f05b965c",
    "measurementId": "G-Bxxxxxx"
    }  # Web App param from firebase and also added databaseURL from Realtime Database Data URL 

In [ ]:
firebase = pyrebase.initialize_app(firebaseConfig)

In [ ]:
realtime_db = firebase.database()

In [ ]:
data = {"name":"John", "age":25, "adress":["new york","los angeles"]}

In [ ]:
# db.push(data)  # create random name

In [ ]:
#db.child("Test_Data").child("Var").set(data)
realtime_db.child("Test_Data").set(data)

In [ ]:
realtime_db.child("real_test2").child("real_test3").remove()  # delete database data

#### DataFrame Data To Firebase (Bucket, Storage)

##### 1 Way

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
firebaseConfig = {
    "apiKey": "xxxxxyAKKZn1tvQy9bxxxxxx",
    "authDomain": "flashxxxxxapp.com",
    "projectId": "flaxxxxx",
    "storageBucket": "flxxxxxx.appspot.com",
    "messagingSenderId": "7xxxxx20981",
    "appId": "1:71xxxxx77a217f05b965c",
    "measurementId": "GxxxxxDJS"
}

In [3]:
cred   = firebase_admin.credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")
app    = firebase_admin.initialize_app(cred, firebaseConfig)
bucket = storage.bucket(app=app)

In [ ]:
def upload_df(df, data_id):
    """
    Upload a Dataframe as a csv to Firebase Storage
    :return: storage_ref
    """

    # Storage location + extension
    storage_ref = data_id + ".csv"

    # Store locally
    df.to_csv(data_id)

    # Upload to Firebase Storage
    blob = bucket.blob(storage_ref)
    with open(data_id,'rb') as local_file:
        blob.upload_from_file(local_file)

    # Delete locally
    os.remove(data_id)

    return storage_ref

In [ ]:
df_test = pd.read_csv("Test.csv")
df_test

In [ ]:
# upload readed file with func
upload_df(df_test, "bucket_test")

In [6]:
# upload file directly without read
bucket.blob(blob_name="test4").upload_from_filename("/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Quran/Flash Cards/ReDe/Test.csv")
# blob_name firebase storage name

##### 2 Way

In [ ]:
import os   # pyrebase running on colab. Tested in colab
import pyrebase
import pandas as pd

In [ ]:
firebaseConfig = {
    "apiKey": "xxxxxxxxZn1tvQy9xxxxxx4",
    "authDomain": "xxxxxardxxxxxp.com",
    "databaseURL": "httpsxxxxxc46e-default-rtdxxxxxxxxxxp/",
    "projectId": "flxxxxxxxxx",
    "storageBucket": "flxxxxxxxxt.com",
    "messagingSenderId": "7xxxxxx981",
    "appId": "1:xxxx120xxxxxx05b965c",
    "measurementId": "GxxxxxxxxS",
    "serviceAccount":"/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json"
    }  # Web App param from firebase and added databaseURL from Realtime Database Data URL and also added serviceAccount

In [ ]:
firebase = pyrebase.initialize_app(firebaseConfig)
storage = firebase.storage()

In [ ]:
df = pd.read_csv("/content/Test.csv")
df

In [ ]:
fileName = "FuturePrices.csv"

In [ ]:
# upload file directly without read
storage.child("fileName").put("/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Quran/Flash Cards/ReDe/Test.csv")